# Setup for Amazon Bedrock
* Container: `conda_python3` <BR>
* We recommend `python 3.10` or later. 
    - version check: !python -V

## 0. Materials
- Bedrock user guide
    - https://docs.aws.amazon.com/bedrock/latest/userguide/what-is-service.html
- Step by step vidio tutorial
    - https://www.youtube.com/watch?v=ab1mbj0acDo

## 1. role setting (adding trust relationship)

### 1.1. role check

In [2]:
from sagemaker import get_execution_role

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [3]:
strSageMakerRoleName = get_execution_role().rsplit('/', 1)[-1]
print (f"SageMaker Execution Role Name: {strSageMakerRoleName}")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
SageMaker Execution Role Name: AmazonSageMaker-ExecutionRole-20221206T163436


### 1.2. policy
- 1.1에서 확인된 롤에 아래와 같이 2개의 권한 추가

![nn](../imgs/policy.jpg)

### 1.3. policy
- 아래와 같이 인라인 정책 생성 (Add permissions - Create inline policy)
- policy name: bedrock
![nn](../imgs/inline-policy.jpg)

In [23]:
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Action": [
                "bedrock:*"
            ],
            "Resource": "*",
            "Effect": "Allow",
            "Sid": "BedrockFullAccess"
        }
    ]
}

{'Version': '2012-10-17',
 'Statement': [{'Action': ['bedrock:*'],
   'Resource': '*',
   'Effect': 'Allow',
   'Sid': 'BedrockFullAccess'}]}

### 1.4. Trust relationship
![nn](../imgs/trust-relationship.jpg)

In [24]:
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Service": "sagemaker.amazonaws.com"
            },
            "Action": "sts:AssumeRole"
        },
        {
            "Effect": "Allow",
            "Principal": {
                "Service": "bedrock.amazonaws.com"
            },
            "Action": "sts:AssumeRole"
        }
    ]
}

{'Version': '2012-10-17',
 'Statement': [{'Effect': 'Allow',
   'Principal': {'Service': 'sagemaker.amazonaws.com'},
   'Action': 'sts:AssumeRole'},
  {'Effect': 'Allow',
   'Principal': {'Service': 'bedrock.amazonaws.com'},
   'Action': 'sts:AssumeRole'}]}

## 2. Model access

### 2.1. Amazon Bedrock Console
![nn](../imgs/model-access/1.png)

### 2.2. "Get Started"
![nn](../imgs/model-access/2.png)

### 2.3. "Model access" - "Edit"
사용 모델 활성화 창
![nn](../imgs/model-access/3.png)

### 2.3. "Save Changes"
사용할 모델을 활성화 후 저장
![nn](../imgs/model-access/4.png)

## 3. Install python SDK for bedrock

In [4]:
install_needed = True

In [5]:
import sys
import IPython

if install_needed:
    print("installing deps and restarting kernel")
    !{sys.executable} -m pip install -U pip
    !{sys.executable} -m pip install -U awscli
    !{sys.executable} -m pip install -U botocore
    !{sys.executable} -m pip install -U boto3
    !{sys.executable} -m pip install -U sagemaker 
    !{sys.executable} -m pip install -U langchain==0.0.312
    !{sys.executable} -m pip install -U termcolor
    !{sys.executable} -m pip install -U transformers
    !{sys.executable} -m pip install -U librosa
    !{sys.executable} -m pip install -U opensearch-py
    !{sys.executable} -m pip install -U sqlalchemy==2.0.1
    !{sys.executable} -m pip install -U pypdf
    !{sys.executable} -m pip install -U spacy
    !{sys.executable} -m spacy download ko_core_news_md
    !{sys.executable} -m pip install -U ipython

    IPython.Application.instance().kernel.do_shutdown(True)

installing deps and restarting kernel
  Obtaining dependency information for awscli from https://files.pythonhosted.org/packages/f0/12/6e567c34832d436c36fc96cef53e527e4e24700ba3a0fd6c3dbcb12a14a0/awscli-1.29.63-py3-none-any.whl.metadata
  Obtaining dependency information for botocore==1.31.63 from https://files.pythonhosted.org/packages/24/0e/39117ec73ea22e700503b3af1dbab270563d6d6ca862cf572899824e4212/botocore-1.31.63-py3-none-any.whl.metadata
  Obtaining dependency information for s3transfer<0.8.0,>=0.7.0 from https://files.pythonhosted.org/packages/5a/4b/fec9ce18f8874a96c5061422625ba86c3ee1e6587ccd92ff9f5bf7bd91b2/s3transfer-0.7.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 25.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 38.4 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 1.2 MB/s eta 0:00:00eta 0:00:01
  Attempting uninstall: botocore
    Found existing installation: b

## 4. Check setting
모델 리스트 <BR>
![nn](../imgs/check.png)


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
module_path = ".."
sys.path.append(os.path.abspath(module_path))

In [3]:
import boto3
import awscli
import botocore
import langchain
from pprint import pprint
from termcolor import colored
from utils.bedrock import bedrock_info

In [4]:
print(f"langchain version check: {langchain.__version__}")
print(f"boto3 version check: {boto3.__version__}")
print(f"botocore version check: {botocore.__version__}")
print(f"awscli version check: {awscli.__version__}")

langchain version check: 0.0.312
boto3 version check: 1.28.63
botocore version check: 1.31.63
awscli version check: 1.29.63


In [18]:
#is_internal_use = <internal-use> ## if 고객: False, aws직원: True
is_internal_use = True

**Select region: "us-east-1"(M1), "us-west-2"(M2)**

In [19]:
#bedrock_region = <your region> ## "us-east-1" or "us-west-2"
bedrock_region = 'us-east-1'

In [20]:
if bedrock_region == "us-east-1":
    bedrock_config = {
        "region_name":bedrock_region,
        "endpoint_url": f"https://bedrock-runtime.{bedrock_region}.amazonaws.com" if is_internal_use else None
    }
elif bedrock_region == "us-west-2":
    bedrock_config = {
        "region_name":bedrock_region,
        "endpoint_url": f"https://bedrock-runtime.{bedrock_region}.amazonaws.com" if is_internal_use else None
        #"endpoint_url": "https://prod.us-west-2.frontend.bedrock.aws.dev" if is_internal_use else None
    }

In [21]:
bedrock_config

{'region_name': 'us-west-2',
 'endpoint_url': 'https://bedrock-runtime.us-west-2.amazonaws.com'}

In [22]:
if is_internal_use:
    bedrock_client = boto3.client(
        service_name='bedrock-runtime',
        region_name=bedrock_config["region_name"],
        endpoint_url=bedrock_config["endpoint_url"]
    )
else:
    bedrock_client = boto3.client(
        service_name='bedrock-runtime',
        region_name=bedrock_config["region_name"]
    )

In [23]:
print (colored("\n== FM lists ==", "green"))
pprint (bedrock_info.get_list_fm_models())


== FM lists ==
{'Claude-Instant-V1': 'anthropic.claude-instant-v1',
 'Claude-V1': 'anthropic.claude-v1',
 'Claude-V2': 'anthropic.claude-v2',
 'Command': 'cohere.command-text-v14',
 'Jurassic-2-Mid': 'ai21.j2-mid-v1',
 'Jurassic-2-Ultra': 'ai21.j2-ultra-v1',
 'Titan-Embeddings-G1': 'amazon.titan-embed-text-v1',
 'Titan-Text-G1': 'TBD'}


## 5. generation

In [34]:
from langchain.llms.bedrock import Bedrock
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

### Titan

## Titan text모델은 현재 preview 서비스입니다. GA 이후 업데이트 하도록 할께요! (현재 에러발생)

In [35]:
llm = Bedrock(
    model_id=bedrock_info.get_model_id(model_name="Titan-Text-G1"),
    client=bedrock_client,
    model_kwargs={
        "maxTokenCount":512,
        "stopSequences":[],
        "temperature":0,
        "topP":0.9
    }
)

In [36]:
prompt = "Please let us know SageMaker's advantages in 100 words"

In [37]:
#print (colored(llm(prompt), "green"))

### Claude v2

In [38]:
llm = Bedrock(
    model_id=bedrock_info.get_model_id(model_name="Claude-V2"),
    client=bedrock_client,
    model_kwargs={
        "max_tokens_to_sample":512,
        "stop_sequences":["\n\nHuman:"],
        "temperature":0,
        "top_p":0.9
    },
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

In [39]:
prompt = "\n\nHuman: Please let us know SageMaker's advantages in 100 words \n\nAssistant:"

In [40]:
response = llm(prompt)

 Here is a 100 word summary of some of SageMaker's key advantages:

SageMaker makes it easy to build, train, and deploy machine learning models at scale. It provides pre-built algorithms and frameworks as well as fully managed infrastructure to quickly get models into production. Automated hyperparameter tuning saves time by finding the best model configurations. SageMaker Studio provides a visual IDE for the full ML workflow. And SageMaker Neo compiles models for faster inference. The service is fully managed so you don't have to worry about infrastructure management. These capabilities help data scientists be more productive and get models to market faster.

* Long text to examine 100k-input

In [55]:
text_file = "../utils/sample-text/long-text-70k-tokens.txt"
with open(text_file, "r") as file:
    text = file.read()
    
prompt = "".join(["\n\nHuman: 아래 글을 요약해줘\n\n", text, "\n\nAssistant:"])
    
print (f"# characters in prompt: {len(prompt)}")

# characters in prompt: 37557


In [56]:
response = llm(prompt)

 한글은 1443년 세종대왕이 한국어를 표기하기 위해 창제한 표음 문자입니다. 

- 초기에는 자음 17개, 모음 11개로 총 28개 자모이었으나 현재는 자음 14개, 모음 10개로 총 24개를 씁니다.

- 자음과 모음을 조합하여 한글 음절을 이룹니다. 

- 발음기관의 형상을 본뜬 과학적인 문자로, 배우기 쉽고 효율적입니다. 

- 조선 시대에 여러 난관을 겪었으나 꾸준히 발전하였습니다.

- 현대에 들어 널리 사용되고 있으며, 세계 언어학계에서도 한글의 우수성을 인정받고 있습니다.

- 1997년 유네스코 세계기록유산으로 등재되었습니다.